In [1]:
from simtk import unit, openmm
from simtk.openmm import app
import numpy as np
from openeye import oechem
import os, smarty, parmed, openmoltools, pdbfixer 
from openmoltools import forcefield_generators

In [2]:
# Generate the SMIRNOFF ligand Structure object
ifs = oechem.oemolistream(smarty.get_data_filename('molecules/AlkEthOH_c100.mol2'))
mol = oechem.OEMol()
flavor = oechem.OEIFlavor_Generic_Default | oechem.OEIFlavor_MOL2_Default | oechem.OEIFlavor_MOL2_Forcefield
ifs.SetFlavor( oechem.OEFormat_MOL2, flavor)
oechem.OEReadMolecule(ifs, mol )
oechem.OETriposAtomNames(mol)

from smarty.forcefield import ForceField
mol_ff = ForceField(smarty.forcefield_utils.get_data_filename('forcefield/Frosst_AlkEtOH.ffxml'))
mol_top, mol_sys, mol_pos = smarty.forcefield_utils.create_system_from_molecule(mol_ff, mol)
molecule_structure = parmed.openmm.load_topology( mol_top, mol_sys) 

print(molecule_structure)
print(molecule_structure.topology)

<Structure 26 atoms; 1 residues; 25 bonds; parametrized>
<Topology; 1 chains, 1 residues, 26 atoms, 25 bonds>


In [3]:
# Generate protein Stucture object
pdbfilename = os.path.join('input', 'input.pdb')
pdbfile = app.PDBFile(pdbfilename)
forcefield = app.ForceField('amber99sbildn.xml', 'tip3p.xml')
system = forcefield.createSystem( pdbfile.topology )
protein_structure = parmed.openmm.load_topology( pdbfile.topology, system)

print(protein_structure)
print(protein_structure.topology)

<Structure 582 atoms; 35 residues; 589 bonds; parametrized>
<Topology; 1 chains, 35 residues, 582 atoms, 589 bonds>


In [4]:
# Merge structures
structure = protein_structure + molecule_structure
topology = structure.topology

# Concatenate positions arrays
positions_unit = unit.angstroms
positions0_dimensionless = np.array( pdbfile.positions / positions_unit )
positions1_dimensionless = np.array( mol_pos / positions_unit )

coordinates = np.vstack((positions0_dimensionless,positions1_dimensionless))
natoms = len(coordinates)
positions = np.zeros([natoms,3], np.float32)
for index in range(natoms):
    (x,y,z) = coordinates[index]
    positions[index,0] = x
    positions[index,1] = y
    positions[index,2] = z
positions = unit.Quantity(positions, positions_unit)

# Store in Structure object
structure.coordinates = coordinates

# Save to PDB
structure.save('test-pdb.pdb',overwrite=True)

In [5]:
# Solvate with PDBFixer
fixer = pdbfixer.PDBFixer('test-pdb.pdb')
#fixer.findMissingResidues()
#fixer.findMissingAtoms()
#fixer.addMissingAtoms()
#fixer.addMissingHydrogens(7.0)
fixer.addSolvent(padding=10*unit.angstrom, ionicStrength=0.050*unit.molar)

In [6]:
# Load PDBFixer object back to Structure
struct = parmed.openmm.load_topology(fixer.topology, xyz=fixer.positions)
struct

<Structure 19945 atoms; 6491 residues; 13471 bonds; PBC (orthogonal); NOT parametrized>

In [7]:
# Select ligand residue by AmberMask
struct[":Alk"]

<Structure 26 atoms; 1 residues; 0 bonds; PBC (orthogonal); NOT parametrized>

In [8]:
# Remove from ligand from protein Structure
struct.strip(":Alk")
struct

<Structure 19919 atoms; 6490 residues; 13471 bonds; PBC (orthogonal); NOT parametrized>

In [64]:
# Regenerate OpenMM System to parameterize solvent
system = forcefield.createSystem( struct.topology )

# Regenerate parameterized protein structure
protein_structure = parmed.openmm.load_topology( struct.topology, system=system, 
                                                xyz=struct.positions, box=struct.box )

In [65]:
# Remerge with ligand structure
combined_structure = protein_structure + molecule_structure

# Restore initial positions and box dimensions
combined_structure.positions = fixer.positions
combined_structure.box = struct.box

In [66]:
print(combined_structure.box)

[ 59.089  59.089  59.089  90.     90.     90.   ]


In [67]:
# Regenerate OpenMM system with ParmEd
combined_structure.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=10.0*unit.angstroms, 
                                constraints=app.HBonds)

AttributeError: 'NoneType' object has no attribute 'req'

In [70]:
combined_structure.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=10.0*unit.angstroms)

AttributeError: 'NoneType' object has no attribute 'req'